# Deep Hedging in Incomplete Markets — GBM & Heston

**MSc Thesis Experiment Runner**

Runs the full deep hedging pipeline under two market dynamics:
- **GBM** (constant volatility, calibrated to S&P 500)
- **Heston** (stochastic volatility, calibrated to S&P 500)

## Setup
1. **Runtime → Change runtime type → A100 GPU** (Pro+ recommended)
2. Click **Connect**
3. Run **Cell 1** (clone + install)

## Two ways to run
- **Option A (Browser):** Run cells directly in this notebook
- **Option B (VS Code):** Run Cell 2 to get an SSH tunnel, then connect VS Code and use the terminal

## Checkpoint / Resume
All progress is automatically checkpointed (Optuna trials in SQLite, per-seed metrics, cached features). If the runtime disconnects mid-run:
1. Reconnect and re-run **Cell 1** (re-clone + install)
2. Re-run the **same experiment cell** — it skips completed work and picks up where it left off

## Safe practice for long runs
For multi-hour or overnight training:
- **Print/log progress regularly.** The pipeline prints each Optuna trial, seed, and stage as it completes — watch the output to monitor progress.
- **Save checkpoints to Google Drive.** Copy the `outputs/` directory to Drive periodically so results survive runtime recycling (see Cell 2b below).
- **Assume sessions can still end and plan to resume.** Even with Pro+, Colab may reclaim GPUs after ~12 hours. The checkpoint system ensures no work is lost — just reconnect and re-run.

In [ ]:
# Cell 1: Clone repo and install dependencies
!git clone https://github.com/thabangTheActuaryCoder/deep-hedging-thesis.git
%cd deep-hedging-thesis
!pip install -q torch numpy matplotlib optuna sqlalchemy

import torch
print(f'\nPython: {__import__("sys").version}')
print(f'PyTorch: {torch.__version__}')
print(f'GPU available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'Device: {torch.cuda.get_device_name(0)}')
    mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f'Memory: {mem:.1f} GB')

In [ ]:
# Cell 2b: Backup & Restore — GitHub (primary) + local + Google Drive (Colab)
# Works on both laptop and Colab. GitHub is the single source of truth.
# Local backup: ~/deep_hedging_backup (survives repo re-clone)

import shutil, os, subprocess, datetime, pathlib

# ── Auto-detect environment ─────────────────────────────────────────
ON_COLAB = os.path.exists('/content')
REPO_DIR = '/content/deep-hedging-thesis' if ON_COLAB else os.getcwd()
LOCAL_OUTPUTS = os.path.join(REPO_DIR, 'outputs')
LOCAL_BACKUP = os.path.join(str(pathlib.Path.home()), 'deep_hedging_backup')
GH_BRANCH = 'experiment-outputs'

# ── Google Drive (Colab only) ──────────────────────────────────────
DRIVE_BACKUP = None
if ON_COLAB:
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        DRIVE_BACKUP = '/content/drive/MyDrive/deep_hedging_outputs'
        print('Google Drive mounted.')
    except Exception as e:
        print(f'Google Drive unavailable: {e}')

# ── GitHub token ───────────────────────────────────────────────────
# Colab: add GITHUB_TOKEN in Colab Secrets (key icon)
# Laptop: export GITHUB_TOKEN=ghp_... in your shell
_gh_token = os.environ.get('GITHUB_TOKEN', '')
if not _gh_token:
    try:
        from google.colab import userdata
        _gh_token = userdata.get('GITHUB_TOKEN')
    except Exception:
        pass

if _gh_token:
    _repo_url = f'https://{_gh_token}@github.com/thabangTheActuaryCoder/deep-hedging-thesis.git'
    subprocess.run(['git', 'remote', 'set-url', 'origin', _repo_url],
                   cwd=REPO_DIR, capture_output=True)
    subprocess.run(['git', 'config', 'user.email', 'experiment@deep-hedging.run'],
                   cwd=REPO_DIR, capture_output=True)
    subprocess.run(['git', 'config', 'user.name', 'Experiment Runner'],
                   cwd=REPO_DIR, capture_output=True)
    print(f'GitHub configured (backup branch: {GH_BRANCH})')
else:
    print('No GITHUB_TOKEN found — GitHub backup disabled.')
    print('  Set it: export GITHUB_TOKEN=ghp_...')


# ── Backup functions ────────────────────────────────────────────────

def backup_to_local():
    """Copy outputs/ to ~/deep_hedging_backup (survives repo re-clone)."""
    if not os.path.exists(LOCAL_OUTPUTS):
        print('No outputs to back up.'); return
    if os.path.exists(LOCAL_BACKUP):
        shutil.rmtree(LOCAL_BACKUP)
    shutil.copytree(LOCAL_OUTPUTS, LOCAL_BACKUP)
    n_files = sum(len(f) for _, _, f in os.walk(LOCAL_OUTPUTS))
    print(f'Backed up {n_files} files to {LOCAL_BACKUP}')


def backup_to_github(message=None):
    """Commit outputs/ to the experiment-outputs orphan branch and push."""
    if not _gh_token:
        print('GitHub backup skipped — no GITHUB_TOKEN set.'); return
    if not os.path.exists(LOCAL_OUTPUTS):
        print('No outputs to back up.'); return

    ts = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
    msg = message or f'Backup outputs {ts}'

    cur_branch = subprocess.run(
        ['git', 'rev-parse', '--abbrev-ref', 'HEAD'],
        cwd=REPO_DIR, capture_output=True, text=True
    ).stdout.strip()

    remote_check = subprocess.run(
        ['git', 'ls-remote', '--heads', 'origin', GH_BRANCH],
        cwd=REPO_DIR, capture_output=True, text=True
    )
    branch_exists = GH_BRANCH in remote_check.stdout

    if branch_exists:
        subprocess.run(['git', 'fetch', 'origin', GH_BRANCH], cwd=REPO_DIR, capture_output=True)
        subprocess.run(['git', 'checkout', GH_BRANCH], cwd=REPO_DIR, capture_output=True)
        subprocess.run(['git', 'reset', '--hard', f'origin/{GH_BRANCH}'], cwd=REPO_DIR, capture_output=True)
    else:
        subprocess.run(['git', 'checkout', '--orphan', GH_BRANCH], cwd=REPO_DIR, capture_output=True)
        subprocess.run(['git', 'rm', '-rf', '.'], cwd=REPO_DIR, capture_output=True)

    out_dest = os.path.join(REPO_DIR, 'outputs')
    if os.path.exists(out_dest) and os.path.realpath(out_dest) != os.path.realpath(LOCAL_OUTPUTS):
        shutil.rmtree(out_dest)
        shutil.copytree(LOCAL_OUTPUTS, out_dest)

    subprocess.run(['git', 'add', 'outputs/'], cwd=REPO_DIR, capture_output=True)
    result = subprocess.run(
        ['git', 'commit', '-m', msg],
        cwd=REPO_DIR, capture_output=True, text=True
    )
    if result.returncode != 0 and 'nothing to commit' in (result.stdout + result.stderr):
        print('GitHub: no new changes to push.')
    else:
        push = subprocess.run(
            ['git', 'push', '-u', 'origin', GH_BRANCH, '--force'],
            cwd=REPO_DIR, capture_output=True, text=True
        )
        if push.returncode == 0:
            n_files = sum(len(f) for _, _, f in os.walk(LOCAL_OUTPUTS))
            print(f'Backed up {n_files} files to GitHub (branch: {GH_BRANCH})')
        else:
            print(f'GitHub push failed: {push.stderr.strip()}')

    subprocess.run(['git', 'checkout', cur_branch], cwd=REPO_DIR, capture_output=True)
    subprocess.run(['git', 'checkout', '.'], cwd=REPO_DIR, capture_output=True)


def backup_to_drive():
    """Copy outputs/ to Google Drive (Colab only)."""
    if not DRIVE_BACKUP:
        print('Google Drive not available (not on Colab).'); return
    if not os.path.exists(LOCAL_OUTPUTS):
        print('No outputs to back up.'); return
    if os.path.exists(DRIVE_BACKUP):
        shutil.rmtree(DRIVE_BACKUP)
    shutil.copytree(LOCAL_OUTPUTS, DRIVE_BACKUP)
    n_files = sum(len(f) for _, _, f in os.walk(LOCAL_OUTPUTS))
    print(f'Backed up {n_files} files to Google Drive')


def backup():
    """Back up to all available destinations."""
    backup_to_local()
    backup_to_github()
    if DRIVE_BACKUP:
        backup_to_drive()


def restore_from_backup():
    """Restore outputs/ — tries GitHub, then local backup, then Drive."""
    if os.path.exists(LOCAL_OUTPUTS):
        n = sum(len(f) for _, _, f in os.walk(LOCAL_OUTPUTS))
        print(f'Outputs already exist locally ({n} files), skipping restore.')
        return True

    # 1. Try GitHub
    if _gh_token:
        print('Checking GitHub for backup...')
        remote_check = subprocess.run(
            ['git', 'ls-remote', '--heads', 'origin', GH_BRANCH],
            cwd=REPO_DIR, capture_output=True, text=True
        )
        if GH_BRANCH in remote_check.stdout:
            subprocess.run(['git', 'fetch', 'origin', GH_BRANCH],
                         cwd=REPO_DIR, capture_output=True)
            result = subprocess.run(
                ['git', 'checkout', f'origin/{GH_BRANCH}', '--', 'outputs/'],
                cwd=REPO_DIR, capture_output=True, text=True
            )
            if result.returncode == 0 and os.path.exists(LOCAL_OUTPUTS):
                n = sum(len(f) for _, _, f in os.walk(LOCAL_OUTPUTS))
                subprocess.run(['git', 'reset', 'HEAD', 'outputs/'],
                             cwd=REPO_DIR, capture_output=True)
                print(f'Restored {n} files from GitHub (branch: {GH_BRANCH})')
                return True

    # 2. Try local backup (~/deep_hedging_backup)
    if os.path.exists(LOCAL_BACKUP):
        print(f'Restoring from {LOCAL_BACKUP}...')
        shutil.copytree(LOCAL_BACKUP, LOCAL_OUTPUTS)
        n = sum(len(f) for _, _, f in os.walk(LOCAL_OUTPUTS))
        print(f'Restored {n} files from local backup')
        return True

    # 3. Try Google Drive
    if DRIVE_BACKUP and os.path.exists(DRIVE_BACKUP):
        print('Restoring from Google Drive...')
        shutil.copytree(DRIVE_BACKUP, LOCAL_OUTPUTS)
        n = sum(len(f) for _, _, f in os.walk(LOCAL_OUTPUTS))
        print(f'Restored {n} files from Google Drive')
        return True

    print('No backup found.')
    return False


# ── Auto-restore on cell run ───────────────────────────────────────
restore_from_backup()
env = 'Colab' if ON_COLAB else 'Local'
print(f'\n[{env}] Backup functions ready:')
print(f'  backup()            — all destinations')
print(f'  backup_to_local()   — {LOCAL_BACKUP}')
print(f'  backup_to_github()  — GitHub (branch: {GH_BRANCH})')
if DRIVE_BACKUP:
    print(f'  backup_to_drive()   — Google Drive')

In [ ]:
# Cell 3: Sanity check — all tests should pass
!python -m pytest tests/test_validation.py -v

In [ ]:
# Cell 4 (QUICK TEST): ~10 min on A100, verifies both GBM + Heston pipelines
!python run_experiment.py --quick --market_model both

In [ ]:
# Cell 5 (FULL RUN): both GBM + Heston, 100k paths
# Safe to re-run after disconnect — automatically resumes from checkpoints
# n_trials=108 = exhaustive search over all configs (dedup skips repeats)
!python run_experiment.py \
    --paths 100000 \
    --N 200 \
    --epochs 1000 \
    --patience 15 \
    --batch_size 2048 \
    --n_trials 108 \
    --seeds 0 1 2 3 4 \
    --substeps 0 5 10 \
    --market_model both

# Auto-backup when experiment finishes (requires Cell 2b)
try:
    backup()
except NameError:
    print('Tip: run Cell 2b first to enable automatic backups')

In [ ]:
# Cell 6: Preview GBM validation plots
from IPython.display import Image, display
import glob

print('=== GBM Validation Plots ===')
for img in sorted(glob.glob('outputs/gbm/plots_val/*.png')):
    print(f'\n--- {img} ---')
    display(Image(filename=img, width=700))

In [ ]:
# Cell 7: Preview Heston validation plots
from IPython.display import Image, display
import glob

print('=== Heston Validation Plots ===')
for img in sorted(glob.glob('outputs/heston/plots_val/*.png')):
    print(f'\n--- {img} ---')
    display(Image(filename=img, width=700))

In [ ]:
# Cell 8: Heston stochastic volatility diagnostic plots
from IPython.display import Image, display
import glob

print('=== Heston Stochastic Volatility Diagnostics ===')
for img in sorted(glob.glob('outputs/heston/plots_heston/*.png')):
    print(f'\n--- {img} ---')
    display(Image(filename=img, width=700))

In [ ]:
# Cell 9: GBM vs Heston comparison plots
from IPython.display import Image, display
import glob

print('=== GBM vs Heston Comparison ===')
for img in sorted(glob.glob('outputs/comparison/*.png')):
    print(f'\n--- {img} ---')
    display(Image(filename=img, width=700))

In [ ]:
# Cell 10: 3D delta surface plots
from IPython.display import Image, display
import glob

for label, pattern in [('GBM', 'outputs/gbm/plots_3d/*.png'),
                       ('Heston', 'outputs/heston/plots_3d/*.png')]:
    imgs = sorted(glob.glob(pattern))
    if imgs:
        print(f'\n=== {label} 3D Delta Surfaces ===')
        for img in imgs:
            print(f'\n--- {img} ---')
            display(Image(filename=img, width=700))

In [ ]:
# Cell 11: Show validation metrics (both market models)
import json, os

for market in ['gbm', 'heston']:
    path = f'outputs/{market}/val_metrics.json'
    if not os.path.exists(path):
        continue
    with open(path) as f:
        metrics = json.load(f)
    print(f'\n{"="*50}')
    print(f'  {market.upper()} — Best model: {metrics["best_model"]}')
    print(f'{"="*50}')
    for model, agg in metrics['aggregated_val_metrics'].items():
        cvar = agg['CVaR95_shortfall']
        mse = agg['MSE']
        print(f'  {model:6s}  CVaR95 = {cvar["mean"]:.6f} +/- {cvar["std"]:.6f}  '
              f'MSE = {mse["mean"]:.6f} +/- {mse["std"]:.6f}')

summary_path = 'outputs/metrics_summary.json'
if os.path.exists(summary_path):
    with open(summary_path) as f:
        combined = json.load(f)
    print(f'\n{"="*50}')
    print('  COMBINED SUMMARY')
    print(f'{"="*50}')
    for market, agg in combined.items():
        print(f'\n  [{market.upper()}]')
        for model, m in agg.items():
            cvar = m.get('CVaR95_shortfall', {})
            if isinstance(cvar, dict):
                print(f'    {model:6s}  CVaR95 = {cvar.get("mean",0):.6f} +/- {cvar.get("std",0):.6f}')

In [ ]:
## Replot (optional)

If you want to adjust figure dimensions, colors, grids, or fonts without
rerunning the experiment, edit the `STYLE` dict in `replot.py` and rerun
Cell 13 below. The data was saved during the experiment.

# Cell 13: Regenerate comparison plots from saved data (edit STYLE in replot.py first)
!python replot.py --data outputs/comparison/comparison_data.pt \
                  --metrics outputs/metrics_summary.json \
                  --out outputs/comparison

from IPython.display import Image, display
import glob
for img in sorted(glob.glob('outputs/comparison/*.png')):
    print(f'\n--- {img} ---')
    display(Image(filename=img, width=700))

In [ ]:
# Cell 14: Download all outputs as zip
import shutil
from google.colab import files

shutil.make_archive('outputs', 'zip', '.', 'outputs')
files.download('outputs.zip')